When discussing the properties of a given atom, it can be tricky to represent all the relevant quantum-mechanical information. In this section, we will explore a common convention for compactly representing an atom's quantum numbers.

In [231]:
import ASDtools as asdt

## **What are term symbols?**

In quantum mechanics, there are **<a href="https://chem.libretexts.org/Bookshelves/Physical_and_Theoretical_Chemistry_Textbook_Maps/Supplemental_Modules_(Physical_and_Theoretical_Chemistry)/Quantum_Mechanics/10%3A_Multi-electron_Atoms/Quantum_Numbers_for_Atoms" target="_blank">four quantum numbers that completely describe an atom</a>**.

+ Principle Quantum Number <br>
+ Orbital Angular Momentum <br>
+ Magnetic Quantum Number <br>
+ Electron Spin Quantum Number <br>

Although we could technically explicitly write out each number for every atomic configuration, term symbols provide a systematic and brief way to describe all four numbers. If you have never encountered term symbols before, please read **<a href="https://chem.libretexts.org/Courses/Pacific_Union_College/Quantum_Chemistry/08%3A_Multielectron_Atoms/8.08%3A_Term_Symbols_Gives_a_Detailed_Description_of_an_Electron_Configuration" target="_blank">this helpful page</a>** that provides an in-depth explanation of them. Generally, term symbols take the following form...

$$ {\Huge ^\text{2S+1}\text{L}_\text{J}} $$

where **S** is the multiplicity, **L** is the total orbital angular momentum, and **J** is the total angular momentum. We need to find all possible values of the above quantum numbers to construct a term symbol for an electronic configuration. Assuming there are _n_ possible ways an electron could be positioned in a subshell, and there are _k_ available electrons, the total number of unique configurations (for a single subshell) is given by the binomial coefficient...

\begin{equation}
\binom{n}{k} = \frac{n!}{k!(n-k)!}.
\end{equation}

However, some of these configurations will produce the same term symbols. This raises an important point: **"term symbols are not an exact description of how electrons are configured in a subshell."** So, how do we find all the possible term symbols associated with a given electronic configuration? I chose to implement a method outlined in **<a href="https://en.wikipedia.org/wiki/Term_symbol" target="_blank">this Wikipedia article</a>**, and checked the output against an **<a href="https://umop.net/spectra/term_calc.php?config=&e=B+&i=0" target="_blank">online calculator</a>**. First, we need to write out the value of ML and MS for each possible electron configuration. Let's walk through an example for non-ionized Carbon (1s2.2s2.2p2). Below is a table that finds ML and MS for all possible configurations.

$$
\begin{aligned}
&\begin{array}{|ccc|cc|}
\hline
\textbf { +1 } & \textbf { 0 } & \textbf { -1 } & \text { $M_L$ } & \text { $M_S$ }\\
\hline
\uparrow & \uparrow & & 1 & 1\\
\uparrow & & \uparrow & 0 & 1\\
 & \uparrow & \uparrow & -1 & 1\\
\downarrow & \downarrow & & 1 & -1\\
\downarrow & & \downarrow & 0 & -1\\
 & \downarrow & \downarrow & -1 & -1\\
\uparrow \downarrow & & & 2 & 0\\
\uparrow & \downarrow & & 1 & 0\\
\uparrow & & \downarrow & 0 & 0\\
\downarrow & \uparrow & & 1 & 0\\
 & \uparrow \downarrow& & 0 & 0\\
 & \uparrow & \downarrow & -1 & 0\\
\downarrow & & \uparrow & 0 & 0\\
 & \downarrow & \uparrow & -1 & 0\\
 & & \uparrow \downarrow & -2 & 0\\
\hline
\end{array}
\end{aligned}
$$

If we count up the number of configurations that produce each ML and MS, we get the following matrix.

$$
\begin{aligned}
&\begin{array}{|c|ccc|}
\hline
& \textbf { +1 } & \textbf { 0 } & \textbf { -1 }\\
\hline
\textbf { +2 } & 0 & 1 & 0 \\
\textbf { +1 } & 1 & 2 & 1 \\
\textbf { 0 } & 1 & 3 & 1 \\
\textbf { -1 } & 1 & 2 & 1 \\
\textbf { -2 } & 0 & 1 & 0 \\
\hline
\end{array}
\end{aligned}
$$

Each unique term symbol is represented by a sub-matrix of all ones, meaning we can calculate each possible term symbol by finding the minimal set of sub-matrices that comprise the original state matrix. __ASDtools__ achieves this by iteratively finding the largest non-zero submatrix within a calculated state matrix. Then, it converts these sub-matrices into a dataframe of term symbols that are displayed using iPython. An example output is shown below.

In [242]:
asdt.generate_term_symbols("1s2.2s2.2p2")

Term Symbols:
-------------


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## **A Small Wrinkle**
Technically, the method outlined above only applies under a handful of assumptions...

1. Only one subhsell is unfilled
2. LS-coupling is the appropriate schema for a given configuration

The first of these are easy to address with a small modification to the method outlined above. If we allow multiple subshells to remain unfilled, then the total number of configurations is given by... 

\begin{equation}
\binom{n_1}{k_1} * \binom{n_2}{k_2} = \frac{n_1!}{k_1!(n_1-k_1)!} * \frac{n_2!}{k_2!(n_2-k_2)!}
\end{equation}

where *k* and *n* are specific to each unfilled subshell. By calculating the quantum numbers for each configuration across all unfilled subshells, we can produce a state matrix that works the same way as above. The function from above automatically handles this extra bit of logic.

In [256]:
asdt.generate_term_symbols("1s2.2s1.2d2")

Term Symbols:
-------------


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

The assumption about LS-coupling is much trickier to account for. For the majority of NIST's spectral lines, this assumption is accurate, so the above function should work. However, for some lines, the reported term symbol uses a different formatting that corresponds with a different coupling schema. Currently, **ASDtools** has no way of producing term symbols in these situations.